# Build a basic ME model

We will try to build an ME model from the NC_000913.2 Genbank file, the iJO1366 M model, and the complex reconstruction from iJL1650-ME

In [ ]:
import re

import cobra.test

# ecoli me
import ecolime
from ecolime.flat_files import *
from ecolime.ecoli_k12 import *


from minime import *
from minime.util import building
from minime.solve.algorithms import binary_search, fva, solve_at_growth_rate
from minime.solve.symbolic import compile_expressions

In [ ]:
building.prepare_for_minime()

In [ ]:
me = MEmodel('iJO1366-ME')
m_model = ecolime.get_m_model()
# some of the "metabolites" in iJO1366 "M" model are actually complexes. We pass those in
# so they get created as complexes, not metabolites.
complex_list = []
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("CPLX"))
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("EG"))
complex_list.extend(i.id for i in m_model.metabolites if "-MONOMER" in i.id)
complex_list.extend(i.id for i in m_model.metabolites if "-CPLX" in i.id)
complex_list.extend(i.id for i in m_model.metabolites if "_mod_" in i.id)
# temp fix
complex_list.extend(i.id for i in m_model.metabolites if i.id.startswith("Isc"))
complex_list = set(complex_list)
building.add_m_model_content(me, m_model, complex_metabolite_ids=complex_list)

# if the bounds of this metabolite aren't open, model uses wrong reactions
me.reactions.EX_pqq_e.lower_bound = -1000

Sometimes multiple entities can perform the same role. To prevent a combinatorial explosion of possibilities, we can create  "generic" version, where any of those entities can fill in.

Create MetabolicReactionData for each iJO1366 reaction

In [ ]:
gb_filename = join(ecoli_files_dir,'NC_000913.2.gb')                                                                                
building.build_reactions_from_genbank(me, gb_filename)

In [ ]:
building.add_dummy_reactions(me, "ATG" + "TTT" * 12 + "TAT" * 12 + "ACG" * 12 + "GAT" * 12 + "AGT" * 12 + "TAA")

### Add in the complex formation

In [ ]:
rna_components = {"b3123"}
complex_stoich_dict = get_complex_to_bnum_dict(rna_components)

In [ ]:
building.add_complex_stoichiometry_data(me, complex_stoich_dict)
for cplx_data in me.complex_data:
    formation = cplx_data.formation
    if formation:
        formation.update()
    else:
        cplx_data.create_complex_formation()

In [ ]:
reaction_to_complex = get_reaction_to_complex(modifications=False)
reaction_info = get_reaction_info_frame()
for reaction_data in me.stoichiometric_data:
    if reaction_data.id.startswith('suf_') or reaction_data.id.startswith('isc_'):
        print reaction_data.id, 'Skipped'
        continue
    building.add_metabolic_reactions(me, reaction_data, reaction_to_complex, reaction_info,
                                     update=True, create_new=True)

In [ ]:
for r in me.reactions:
    if hasattr(r, "update"):
        r.update()

In [ ]:
for c_d in me.complex_data:
    c = c_d.complex
    if len(c.reactions) == 1:
        list(c.reactions)[0].delete(remove_orphans=True)
for p in me.metabolites.query("protein"):
    if len(p._reaction) == 1:
        list(p._reaction)[0].delete(remove_orphans=True)
for m in me.metabolites.query("RNA"):
    delete = True
    for reaction in m._reaction:
        if isinstance(reaction, TranslationReaction):
            delete = False
    if delete:
        while len(m._reaction) > 0:
            list(m._reaction)[0].delete(remove_orphans=True)
for rxn in me.reactions.query('transcription_'):
     if len(rxn.products) < 3:
            rxn.remove_from_model(remove_orphans=True)

In [ ]:
me_genes = set([i.id.replace('RNA_','') for i in me.metabolites.query("RNA_b")])

In [ ]:
import cobra.test
iJO = cobra.test.create_test_model('ecoli')
iJO_genes = set([i.id for i in iJO.genes])

In [ ]:
# These differences are iron_sulfur proteins
me_genes.difference(iJO_genes)

In [ ]:
# I think these are just tRNA synthetases
iJO_genes.difference(me_genes)

In [ ]:
len(me.reactions.query('transcription'))

In [ ]:
len(me.reactions)

# Solve

In [ ]:
divalent_list = divalent_list
monovalent_list = ['_mod_k','_mod_na1']
from pickle import load
with open("test_keffs.pickle", "rb") as infile:
    old_keffs = load(infile)
keffs = {}

for keff, value in old_keffs.items():
    for i in divalent_list: 
        keff = keff.replace(i, 'generic_divalent')
    for i in monovalent_list: 
        keff = keff.replace(i, '_mod_generic_monovalent')
    keffs[keff] = value
    
for r in me.reactions:
    if isinstance(r, MetabolicReaction) and r.complex_data is None:
        continue
    if isinstance(r, MetabolicReaction) and r.complex_data.id != "CPLX_dummy":
        met_rxn = r
        key = met_rxn.id.replace("-", "_DASH_").replace("__", "_DASH_").replace(":","_COLON_")
        #key = met_rxn.id
        key = "keff_" + key.replace("_FWD_", "_").replace("_REV_", "_")

        matches = [i for i in keffs if key in i]
        # get the direction
        if met_rxn.reverse:
            matches = [i for i in matches if i.endswith("_reverse_priming_keff")]
        else:
            matches = [i for i in matches if i.endswith("_forward_priming_keff")]
        if len(matches) == 1:
            met_rxn.keff = keffs[matches[0]]
        elif len(matches) > 0:
            if len(matches) == len([i for i in matches if key + "_mod_"]):
                met_rxn.keff = keffs[matches[0]]
            else:
                print key, len(matches)
        else:  # len(matches) == 0
            print "no keff found for", key

In [ ]:
me.reactions.dummy_reaction_FWD_CPLX_dummy.objective_coefficient = 1.

In [ ]:
# Turn off reactions that throw off results as in iOL
KO_list = ['DHPTDNR','DHPTDNRN', 'SUCASPtpp','SUCFUMtpp', 'SUCMALtpp', 'SUCTARTtpp', 
           'CAT', 'FHL', 'SPODM', 'SPODMp']
for reaction in KO_list:
    a = me.reactions.query(reaction + '_')
    for rxn in a:
        rxn.upper_bound = 0
        rxn.lower_bound = 0

In [ ]:
me.reactions.EX_glc__D_e.lower_bound = -10

In [ ]:
expressions = compile_expressions(me)

In [ ]:
binary_search(me, min_mu=.05, max_mu=2.1, mu_accuracy=1e-2,
              compiled_expressions=expressions)

In [ ]:
me.metabolites.kdo2lipid4_e.reactions

In [ ]:
me.metabolites.get_by_id('2agpe160_p').name

In [ ]:
me.metabolites.RNA_b0007.reactions